# Streaming

Streaming...

There are two types frequently used with Agents:

In [13]:
from langchain.agents import create_agent

In [14]:
agent = create_agent(
    model="openai:gpt-5-mini",
    system_prompt="You are a full-stack comedian",
)

## No Steaming (invoke)

In [15]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

Why did the developer go broke? Because he used up all his cache.


## values

In [16]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a joke
================================== Ai Message ==================================

Why do programmers prefer dark mode?

Because light attracts bugs.

Want another one—front-end, back-end, or full-stack roast?


## message

In [17]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

I wrote you a poem in JavaScript,
but it kept returning undefined,
so I refactored it in coffee—black, no sugar—
and the loop of longing finally broke.

There’s a little server in my chest
that listens on port midnight,
accepts requests with a soft "hello" header,
and responds with an emoji of your name.

I commit small acts of kindness like pushes,
and merge them into the main branch of days.
Sometimes bugs slip through—missing silence,
a race condition of busy schedules—but
I write tests: I call, I knock, I show up.

Your smile is a bugfix that compiles at sunrise,
turns my deprecated fears into legacy comments.
We deploy ourselves on messy afternoons,
rolled back only when laughter demands it.

If love had a stack trace it would point to you:
line one, the breath you borrow from the air;
line two, the coffee shared at half-past too-late;
line three, the quiet where both our cursors blink.

So here’s the README: I like you. I will update.
No permission required, no sudo, just try it.

## Tools can stream too!

In [19]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='fae1ef24-aed9-4e7f-8f5e-cf053497f728')]})
('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='fae1ef24-aed9-4e7f-8f5e-cf053497f728'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 132, 'total_tokens': 222, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CO5C8M1sv5jIo1pawZTaQCrTnkxdT', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--58b5f6cd-416d-479d-9a55-2296963543a8-0', tool_calls=[{'name': 'get_we

In [20]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])

Looking up data for city: San Francisco, CA
Acquired data for city: San Francisco, CA
